In [38]:
%pip install numpy==2.3

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
    --------------------------------------- 0.3/12.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.7 MB 1.2 MB/s eta 0:00:11
   -- ------------------------------------- 0.8/12.7 MB 1.2 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/12.7 MB 1.2 MB/s eta 0:00:11
   ---- ----------------------------------- 1.3/12.7 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.6/12.7 MB 1.2 MB/s eta 0:00:10
   ----- ---------------------------------- 1.8/12.7 MB 1.2 MB/s eta 0:00:10
   ------ --------------------------------- 2.1/12.7 MB 1.2 MB/s eta 0:00:10
   ------- -------------------------------- 2.4/12.7 MB 1.2 MB/s eta 0:00:09
   -------- ------------------------------- 2.6/12.7 MB 1.2 MB/s eta 0:00:09
   --------- ------


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
%pip install numba

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.5-cp313-cp313-win_amd64.whl.metadata (60 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 1.2 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.8 MB 1.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.0/2.8 MB 1.2 MB/s eta 0:00:02
   ------------------- -------------------- 1.3/2.8 MB 1.2 MB/s eta 0:00:02
   ---------------------- ----------------- 1.6/2.8 MB 1.2 MB/s eta 0:00:02
   -------------------------- ------------- 1.8/2.8 MB 1.2 MB/s eta 0:00:01
   ------------------------------ --------- 2.1/2.8 MB 1.2 MB/s eta 0:00:01
   ---------------------------------- ----- 2.4/2.8 MB 1.2 MB/s eta 0:00:01
   ----------------------------------

  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import networkx as nx
import pickle

In [2]:
import numpy as np

In [3]:
import heapq

In [4]:
from numba import jit

In [5]:
import random

In [29]:
import time

In [ ]:
from time import strftime, localtime

In [6]:
import math

In [7]:
K_SEEDS = 40
POP_SIZE = 10
MAX_GEN = 150
P_CROSSOVER = 0.6
P_MUTATION = 0.1
LAMBDA_VAL = 0.5
PROPAGATION_PROB = 0.01
MC_SIMULATIONS = 1000

In [8]:
NUM_RUNS = 20

In [10]:
def live_edge_to_edgelist(live_edge_graphs, target_nodes, p_attend):
    Gs = []
    start_poses = []
    ws = []
    Ps = []
    for h in live_edge_graphs:
        g = nx.DiGraph(h)
        # print("egdes:", g.number_of_edges())
        G = np.zeros((g.number_of_edges(), 2), dtype=np.int32)
        P = np.zeros(g.number_of_edges())
        w = np.zeros(len(g))
        w[list(target_nodes)] = 1
        start_pos = np.zeros(len(g)+1, dtype=np.int32)
        curr_pos = 0
        for v in g:
            start_pos[v] = curr_pos
            for u in g.predecessors(v):
                G[curr_pos] = [u, v]
                P[curr_pos] = p_attend[u]
                curr_pos += 1
        start_pos[-1] = g.number_of_edges()
        Gs.append(G)
        Ps.append(P)
        start_poses.append(start_pos)
        ws.append(w)
    return Gs, Ps, start_poses, ws

In [25]:
@jit
def gradient_coverage_single_edgelist(x, G, P, start_pos, w, v):
    grad = np.zeros((x.shape[0]))
    #process gradient entries one node at a time
    p_all_fail = 1
    for j in range(start_pos[v], start_pos[v+1]):
        p_all_fail *= 1 - x[G[j]]*P[j]
    p_all_fail *= 1 - x[v]
    for j in range(start_pos[v], start_pos[v+1]):
        #0/0 should be 0 here
        if p_all_fail == 0:
            p_others_fail = 0
        else:
            p_others_fail = p_all_fail/(1 - x[G[j]]*P[j])
        grad[G[j]] += w[v]*P[j]*p_others_fail
    if p_all_fail > 0:
        grad[v] += w[v]*p_all_fail/(1 - x[v])
    if(np.any(np.isnan(grad))):
        print(x.min(), x.max())
        raise Exception()
    return grad

In [11]:
@jit
def gradient_estimate_all_nodes(x, Gs, Ps, start_poses, ws, B):
    '''
    Returns a stochastic estimate of the gradient of the probability that 
    every node is influenced wrt every selectable node. 
    '''
    n = start_poses[0].shape[0] - 1
    grad_est = np.zeros((x.shape[0], n))
    for b in range(B):
        v = random.randint(0, n-1)
        idx = random.randint(0, len(Gs)-1)
        grad_est[:, v] += (n/float(B)) * gradient_coverage_single_edgelist(x, Gs[idx], Ps[idx], start_poses[idx], ws[idx], v)
    return grad_est

In [27]:
@jit
def marginal_coverage_edgelist(x, G, P, start_pos, w):
    probs = np.ones(x.shape[0])
    for v in range(x.shape[0]):
        for j in range(start_pos[v], start_pos[v+1]):
            probs[v] *= 1 - x[G[j]]*P[j]
    probs *= 1 - x
    probs = 1 - probs
    return probs

In [26]:
def make_multilinear_gradient_group(live_graphs, group_indicator, target_nodes, selectable_nodes, p_attend):
    Gs, Ps, start_poses, ws = live_edge_to_edgelist(live_graphs, target_nodes, p_attend)    
    def gradient(x, batch_size):
        x_expand = np.zeros(len(live_graphs[0]))
        x_expand[selectable_nodes] = x
        grad = gradient_estimate_all_nodes(x, Gs, Ps, start_poses, ws, batch_size) @ group_indicator
        return grad[selectable_nodes, :]
    return gradient


In [28]:
def make_multilinear_objective_samples_group(live_graphs, group_indicator, target_nodes, selectable_nodes, p_attend):
    Gs, Ps, start_poses, ws = live_edge_to_edgelist(live_graphs, target_nodes, p_attend)
    def f_all(x, batch_size):
        x_expand = np.zeros(len(live_graphs[0]))
        x_expand[selectable_nodes] = x
        pr_reached = np.zeros(len(x_expand))
        for b in range(batch_size):
            i = random.randint(0, len(Gs)-1)
            pr_reached += (1./batch_size)*marginal_coverage_edgelist(x_expand, Gs[i], Ps[i], start_poses[i], ws[i])
        return pr_reached @ group_indicator
    return f_all

In [13]:
def load_data(pickle_file, attribute_name='group'):
    with open(pickle_file, 'rb') as f:
        g = pickle.load(f)
    
    G = g.to_undirected() if g.is_directed() else g

    node_groups = {}
    for node in G.nodes():
        if attribute_name in G.nodes[node]:
            node_groups[node] = G.nodes[node][attribute_name]
            if attribute_name != 'group':
                G.nodes[node]['group'] = G.nodes[node][attribute_name]
        else:
            node_groups[node] = 0
            G.nodes[node]['group'] = 0
    
    return G, node_groups

In [14]:
def indicator(S, n):
    x = np.zeros(n)
    x[list(S)] = 1
    return x

In [15]:
def multi_to_set(f, n = None, g_nodes = None):
    if n == None:
        if g_nodes is not None:
            n = len(g_nodes)
        else:
            raise ValueError("Either n or g_nodes must be provided")
    def f_set(S):
        return f(indicator(S, n))
    return f_set

In [16]:
def greedy(items, mc, f):
    if mc >= len(items):
        S = set(items)
        return S, f(S)
    upper_bounds = [(-f(set([u])), u) for u in items]    
    heapq.heapify(upper_bounds)
    starting_objective = f(set())
    S  = set()
    
    while len(S) < mc:
        val, u = heapq.heappop(upper_bounds)
        new_total = f(S.union(set([u])))
        new_val =  new_total - starting_objective
        if new_val >= -upper_bounds[0][0] - 0.01:
            S.add(u)
            starting_objective = new_total
        else:
            heapq.heappush(upper_bounds, (-new_val, u))
    return S, starting_objective

In [17]:
def sample_live_icm(g, num_graphs):
    live_edge_graphs = []
    for _ in range(num_graphs):
        h = nx.Graph()
        h.add_nodes_from(g.nodes())
        for u,v in g.edges():
            if random.random() < g[u][v]['p']:
                h.add_edge(u,v)
        live_edge_graphs.append(h)
    return live_edge_graphs

In [ ]:
# def greedy_max_influence(G_sub, k, p=0.01, mc=20):
#     S = []

#     if k > len(G_sub):
#         return run_IC(G_sub, list(G_sub.nodes()), p, mc)

#     for _ in range(k):
#         best_node = None
#         max_gain = -1
#         candidates = list(G_sub.nodes())
        
#         # Nếu quá nhiều candidates, sample để tăng tốc
#         if len(candidates) > 100:
#             candidates = random.sample(candidates, 100)

#         for node in candidates:
#             if node in S:
#                 continue
#             spread = run_IC(G_sub, S + [node], p, mc=10)
#             if spread > max_gain:
#                 max_gain = spread
#                 best_node = node
        
#         if best_node:
#             S.append(best_node)
            
#     return run_IC(G_sub, S, p, mc=mc)

In [ ]:
graphname = "rice_subset"

In [ ]:
G, node_groups_map = load_data(f"dataset/{graphname}.pickle", attribute_name='color')

In [19]:
fair_vals_attr = np.zeros((NUM_RUNS, len(set(node_groups_map.values()))))
greedy_vals_attr = np.zeros((NUM_RUNS, len(set(node_groups_map.values()))))

In [20]:
include_total = False

In [1]:
group_size = {}

In [21]:
groups = {}
for n, g in node_groups_map.items():
    if g not in groups: groups[g] = []
    groups[g].append(n)

In [22]:
print(f"Nodes: {len(G)}, Edges: {G.number_of_edges()}")
print(f"Groups: {list(groups.keys())}") 

Nodes: 445, Edges: 9660
Groups: ['0', '1']


In [23]:
ideal_influences = {}
N = len(G)

In [24]:
alpha = 0.5

In [ ]:
def f_multi(x):
  return val_oracle(x, 1000).sum()

In [ ]:
f_set = multi_to_set(lambda x: run_IC(G, np.where(x==1)[0].tolist(), p=PROPAGATION_PROB, mc=MC_SIMULATIONS), n=N)

In [ ]:
for attribute_index, attribute in enumerate(set(node_groups_map.values())):
    live_graphs = sample_live_icm(G, MC_SIMULATIONS)
    group_indicator = np.ones((len(G.nodes()), 1))
    oracle_values = make_multilinear_objective_samples_group(live_graphs, group_indicator, set(G.nodes()), set(G.nodes()), p_attend={u: 1.0 for u in G.nodes()})
    def f_multi(x):
        return oracle_values(x, 1000).sum()
    f_set = multi_to_set(f_multi, n=N)
    violation_0 = []
    violation_1 = []
    min_fraction_0 = []
    min_fraction_1 = []
    pof_0 = []
    time_0 = []
    time_1 = []
    
    for run in range(NUM_RUNS):
        print(strftime("%Y-%m-%d %H:%M:%S"), localtime(), f" - Starting fair run {run} for attribute {attribute}...")
        start_time1 = time.perf_counter()
        S, obj = greedy(list(groups[attribute]), K_SEEDS, f_set)
        end_time1 = time.perf_counter()
        runningtime1 = end_time1 - start_time1
        
        start_time = time.perf_counter()
        values = np.unique([g.nodes[n]['group'] for n in G.nodes()])
        node_attributes = {}
        for vidx, val in enumerate(values):
            node_attributes[val] = [v for v in G.nodes() if g.nodes[v]['group'] == val]
            group_size[graphname][attribute][run, vidx] = len(node_attributes[val])
        
        opt_succession = {}
        
        
        